# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [3]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [4]:
#Read in weather csv from part 1
weather = pd.read_csv("../WeatherPy/output_data/cities_weather.csv")
weather

,City,Country,Date,Lat,Lng,Cloudiness,Humidity,Max Temp,Wind Speed
0,Punta Arenas,CL,1612723302,-53.1500,-70.9167,0,23,69.80,12.66
1,Pozo Colorado,PY,1612724005,-23.4923,-58.7972,0,22,93.76,4.16
2,George Town,MY,1612723872,5.4112,100.3354,20,83,80.60,3.49
3,Adrar,MR,1612724005,20.5022,-10.0711,0,29,63.12,8.61
4,Atuona,PF,1612724005,-9.8000,-139.0333,23,75,79.41,21.63
...,...,...,...,...,...,...,...,...,...
566,Sagua de Tánamo,CU,1612724081,20.5819,-75.2414,73,63,83.25,5.46
567,Chapais,CA,1612724081,49.7834,-74.8492,90,79,15.80,10.36
568,Nuuk,GL,1612723850,64.1835,-51.7216,0,40,33.80,6.91
569,Solnechnyy,RU,1612723497,50.7214,136.6319,100,90,0.48,3.65


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [5]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# Store latitude and longitude in locations
locations = weather[["Lat", "Lng"]]

# Store Humidity in humidity
humidity = weather["Humidity"]

In [6]:
# Plot Heatmap
fig = gmaps.figure(center=(46.0, -5.0), zoom_level=2)
max_intensity = np.max(humidity)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, 
                                 dissipating=False, max_intensity=100, point_radius=3)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [11]:
#Narrow down the DataFrame to find your ideal weather condition. Ideal Conditions: 
#Max temp between 65 and 85
#Wind speed less than 10 mph.
#10% or less cloudiness. 
#Humidity below 50%
#Drop any rows that don't contain all three conditions.
ideal_cities = weather.loc[(weather["Max Temp"] >=70) & (weather["Max Temp"] <85) &
                          (weather["Wind Speed"] <10) & (weather["Cloudiness"] <10)
                          & (weather["Humidity"] < 50)].dropna()
ideal_cities

,City,Country,Date,Lat,Lng,Cloudiness,Humidity,Max Temp,Wind Speed
38,Coyhaique,CL,1612723890,-45.5752,-72.0662,0,35,73.40,6.91
141,Ancud,CL,1612723907,-41.8697,-73.8203,0,34,80.60,6.91
189,Castro,BR,1612724029,-24.7911,-50.0119,0,47,76.24,3.47
315,Berriozábal,MX,1612724046,16.8000,-93.2667,1,39,82.40,9.22
468,Gambēla,ET,1612724066,8.2500,34.5833,0,20,82.89,2.06
530,Dosso,NE,1612724074,13.0490,3.1937,7,17,82.72,8.16
564,La Providencia,MX,1612724080,28.7167,-111.5833,0,32,73.00,1.01


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [12]:
#Eliminate unneeded rows from DF
hotel_df = ideal_cities.loc[:, ["City", "Country", "Lat", "Lng"]]
hotel_df

,City,Country,Lat,Lng
38,Coyhaique,CL,-45.5752,-72.0662
141,Ancud,CL,-41.8697,-73.8203
189,Castro,BR,-24.7911,-50.0119
315,Berriozábal,MX,16.8000,-93.2667
468,Gambēla,ET,8.2500,34.5833
530,Dosso,NE,13.0490,3.1937
564,La Providencia,MX,28.7167,-111.5833


In [13]:
#Add column to hold Hotel Name
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Lat,Lng,Hotel Name
38,Coyhaique,CL,-45.5752,-72.0662,
141,Ancud,CL,-41.8697,-73.8203,
189,Castro,BR,-24.7911,-50.0119,
315,Berriozábal,MX,16.8000,-93.2667,
468,Gambēla,ET,8.2500,34.5833,
530,Dosso,NE,13.0490,3.1937,
564,La Providencia,MX,28.7167,-111.5833,


In [14]:
#Set parameters to search for hotels with 5000 meters.
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

In [15]:
#Hit the Google Places API for each city's coordinates.
#Store the first Hotel result into the DataFrame.
print("-------Start of Search-------")
for index, row in hotel_df.iterrows():
    # get city name, lat, lnt from df
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    
    # add keyword to params dict
    params["location"] = f"{lat},{lng}"

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    # save the hotel name to dataframe
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']

    # if there is no hotel available, show missing field
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

# Print end of search once searching is completed
print("-------End of Search-------")

-------Start of Search-------
Retrieving Results for Index 38: Coyhaique.
Closest hotel in Coyhaique is Entre Cumbres Apart Hotel.
------------
Retrieving Results for Index 141: Ancud.
Closest hotel in Ancud is Panamericana Hotel Ancud.
------------
Retrieving Results for Index 189: Castro.
Closest hotel in Castro is Pousada Rota Dos Tropeiros.
------------
Retrieving Results for Index 315: Berriozábal.
Missing field/result... skipping.
------------
Retrieving Results for Index 468: Gambēla.
Closest hotel in Gambēla is Grand Resort And Spa,Gambella.
------------
Retrieving Results for Index 530: Dosso.
Closest hotel in Dosso is Hôtel Zigi.
------------
Retrieving Results for Index 564: La Providencia.
Missing field/result... skipping.
------------
-------End of Search-------


In [16]:
hotel_df

,City,Country,Lat,Lng,Hotel Name
38,Coyhaique,CL,-45.5752,-72.0662,Entre Cumbres Apart Hotel
141,Ancud,CL,-41.8697,-73.8203,Panamericana Hotel Ancud
189,Castro,BR,-24.7911,-50.0119,Pousada Rota Dos Tropeiros
315,Berriozábal,MX,16.8000,-93.2667,
468,Gambēla,ET,8.2500,34.5833,"Grand Resort And Spa,Gambella"
530,Dosso,NE,13.0490,3.1937,Hôtel Zigi
564,La Providencia,MX,28.7167,-111.5833,


In [17]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [19]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))